In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import sys
sys.path.append('..')
import ScraperFC as sfc # import local scraperfc
import traceback

from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from ScraperFC import xpath_soup
import re
import requests
import pandas as pd
import numpy as np

print('Done importing.')

Done importing.


In [23]:
fbref = sfc.FBRef()
try:
    x = fbref.scrape_league_table(2021, 'EPL')
except:
    traceback.print_exc()
finally:
    fbref.close()

Scraping 2021 EPL league table


In [25]:
x

,Rk,Squad,MP,W,D,L,GF,GA,GD,Pts,Pts/MP,xG,xGA,xGD,xGD/90,Attendance,Top Team Scorer,Goalkeeper,Notes
0,1,Manchester City,38,27,5,6,83,32,51,86,2.26,68.2,30.2,38.0,1.00,526,İlkay Gündoğan - 13,Ederson,→ Champions League via league finish
1,2,Manchester Utd,38,21,11,6,73,44,29,74,1.95,60.1,41.4,18.6,0.49,526,Bruno Fernandes - 18,David de Gea,→ Champions League via league finish
2,3,Liverpool,38,20,9,9,68,42,26,69,1.82,67.5,43.0,24.5,0.65,837,Mohamed Salah - 22,Alisson,→ Champions League via league finish
3,4,Chelsea,38,19,10,9,58,36,22,67,1.76,62.4,30.3,32.1,0.85,526,Jorginho - 7,Edouard Mendy,→ Champions League via league finish
4,5,Leicester City,38,20,6,12,68,50,18,66,1.74,55.8,46.0,9.8,0.26,421,Jamie Vardy - 15,Kasper Schmeichel,→ Europa League via cup win
5,6,West Ham,38,19,8,11,62,47,15,65,1.71,55.4,48.7,6.7,0.18,632,"Michail Antonio, Tomáš Souček - 10",Łukasz Fabiański,→ Europa League via league finish
6,7,Tottenham,38,18,8,12,68,45,23,62,1.63,53.1,49.1,4.0,0.10,632,Harry Kane - 23,Hugo Lloris,→ Europa Conference League via league finish 1
7,8,Arsenal,38,18,7,13,55,39,16,61,1.61,51.8,43.0,8.8,0.23,632,Alexandre Lacazette - 13,Bernd Leno,NaN
8,9,Leeds United,38,18,5,15,62,54,8,59,1.55,55.6,57.9,-2.2,-0.06,421,Patrick Bamford - 17,Illan Meslier,NaN
9,10,Everton,38,17,8,13,47,48,-1,59,1.55,45.7,50.1,-4.5,-0.12,368,Dominic Calvert-Lewin - 16,Jordan Pickford,NaN
